# 比較 LDA topic numbers

參考資料
- https://medium.com/pyladies-taiwan/%E4%BB%A5-jieba-%E8%88%87-gensim-%E6%8E%A2%E7%B4%A2%E6%96%87%E6%9C%AC%E4%B8%BB%E9%A1%8C-%E4%BA%94%E6%9C%88%E5%A4%A9%E4%BA%BA%E7%94%9F%E7%84%A1%E9%99%90%E5%85%AC%E5%8F%B8%E6%AD%8C%E8%A9%9E%E5%88%86%E6%9E%90-ii-fdf5d3708662


In [10]:
import os
import numpy as np
import pandas as pd
from collections import Counter
from enum import Enum
from pprint import pprint
import matplotlib.pyplot as plt

import nltk
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()

import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora, models

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s',
                    level=logging.DEBUG)

## 自定義 data types and functions

In [2]:
class ContentType(Enum):
    TIT = 'title'
    ABS = 'abstract'
    AUT = 'author'
    SEC = 'section'
    
def get_contents(content_type):
    all_contents = []
    dataset_path = '../dataset'
    for file in os.listdir(dataset_path):
        file_path = os.path.join(dataset_path, file)
        if os.path.isfile(file_path):
            with open(file_path) as f:  
                line = f.readlines()
                if content_type == ContentType.AUT:
                    line = line[1]
                elif content_type == ContentType.SEC:
                    line = line[2]
                elif content_type == ContentType.ABS:
                    line = line[3]
                else:
                    line = line[0]
                line = line.strip()
                all_contents.append(line)
        else:
            print(file_path + ' does not exist.')
    return all_contents


def get_all_titles():
    return get_contents(ContentType.TIT)

def get_all_authors():        
    return get_contents(ContentType.AUT)

def get_all_sections():
    return get_contents(ContentType.SEC)

def get_all_abstracts():
    return get_contents(ContentType.ABS)

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            token = wordnet_lemmatizer.lemmatize(token, pos='v')
            token = wordnet_lemmatizer.lemmatize(token, pos='n')
            result.append(token)
    return result

def get_chart_data(num_topics,num_words,topics):
    buff = 300
    K = num_topics
    topicWordProbMat = topics
    
    #columns = ['1','2','3','4','5']
    columns = range(1,num_topics+1)

    df = pd.DataFrame(columns = columns)
    pd.set_option('display.width', 1000)

    # 40 will be resized later to match number of words in DC
    zz = np.zeros(shape=(buff,K))

    last_number = 0
    DC = {}

    for x in range (num_words): #取每個topic前10個字
        data= pd.DataFrame(columns=columns,index=[0])
        for i in range(num_topics):
            data[columns[i]] = ""
        df = df.append(data,ignore_index=True)  

    for line in topicWordProbMat:
        topic_id,words = line #一個line是一個topic
        probs = words.split("+")
        y = 0 #用來算第幾個word
        for pr in probs:    
            a = pr.split("*")
            df.iloc[y,topic_id] = a[1] #該word

            if a[1] in DC:
                zz[DC[a[1]]][topic_id] = a[0] #該word的機率
            else:
                zz[last_number][topic_id] = a[0]
                DC[a[1]] = last_number
                last_number = last_number+1
            y = y + 1

    return (df,DC,zz)

def show_words_table(df):  
    print(df)
    print('\n')
    
def show_dictionary(DC):
    print(DC)
    print('字典字數：',len(DC))
    print('\n')

def show_probs_table(zz):
    print(zz)
    print(zz.shape)
    
def show_heapmap(DC,zz):
    %matplotlib inline

    zz = np.resize(zz,(len(DC.keys()),zz.shape[1]))

    for val, key in enumerate(DC.keys()):
            plt.text(-3.5, val + 0.1, key,
                     horizontalalignment='right',
                     verticalalignment='center'
                     )

    #plt.figure(figsize=(10,50))
    plt.imshow(zz, cmap='hot', interpolation='nearest',aspect=0.5)#'auto'
    plt.show()

### 取出所有摘要

In [3]:
contents = get_all_abstracts()
print('共',len(contents),'篇論文\n')

documents = pd.DataFrame(data=contents,columns=['abstract'])
documents['index'] = documents.index
documents[:10]

共 1343 篇論文



,abstract,index
0,We consider the problem of actively eliciting ...,0
1,We investigate the task of distractor generati...,1
2,The most common representation formalisms for ...,2
3,Statistical relational learning models are pow...,3
4,Multimodal representation learning is gaining ...,4
5,Reinforcement learning (RL) has shown its adva...,5
6,Selecting appropriate tutoring help actions th...,6
7,Recognizing time expressions is a fundamental ...,7
8,"When facing large-scale image datasets, online...",8
9,Temporal modeling in videos is a fundamental y...,9


### 預處理的全部論文摘要

In [4]:
processed_docs = documents['abstract'].map(preprocess)
processed_docs[:10]

0    [consider, problem, actively, elicit, preferen...
1    [investigate, task, distractor, generation, mu...
2    [common, representation, formalism, plan, desc...
3    [statistical, relational, learn, model, powerf...
4    [multimodal, representation, learn, gain, deep...
5    [reinforcement, learn, show, advantage, image,...
6    [select, appropriate, tutor, help, action, acc...
7    [recognize, time, expression, fundamental, imp...
8    [face, large, scale, image, datasets, online, ...
9    [temporal, model, video, fundamental, challeng...
Name: abstract, dtype: object

## Dataset

### 產生字典

In [5]:
dictionary = gensim.corpora.Dictionary(processed_docs)
print('共',len(dictionary),'個字\n')

2019-11-02 22:08:58,566 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2019-11-02 22:08:58,692 : INFO : built Dictionary(6927 unique tokens: ['active', 'actively', 'adaptive', 'aggregation', 'algorithm']...) from 1343 documents (total 121739 corpus positions)


共 6927 個字



### 產生 bag of words corpus

In [6]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
print('共',len(bow_corpus),'筆')

共 1343 筆


### 產生 TF-IDF Corpus

In [7]:
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
print('共',len(corpus_tfidf),'筆')

2019-11-02 22:09:00,981 : INFO : collecting document frequencies
2019-11-02 22:09:00,985 : INFO : PROGRESS: processing document #0
2019-11-02 22:09:01,011 : INFO : calculating IDF weights for 1343 documents and 6926 features (84803 matrix non-zeros)


共 1343 筆


## Train LDA models

In [8]:
num_topics = 10
num_words = 10

### Running LDA using TF-IDF

In [13]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, 
                                       num_topics=num_topics, 
                                       id2word=dictionary, 
                                       passes=20,
                                       iterations=50,
                                       eval_every=1)

2019-11-02 22:10:58,107 : INFO : using symmetric alpha at 0.1
2019-11-02 22:10:58,109 : INFO : using symmetric eta at 0.1
2019-11-02 22:10:58,113 : INFO : using serial LDA version on this node
2019-11-02 22:10:58,128 : INFO : running online LDA training, 10 topics, 20 passes over the supplied corpus of 1343 documents, updating every 6000 documents, evaluating every ~1343 documents, iterating 50x with a convergence threshold of 0.001000
2019-11-02 22:10:58,133 : INFO : training LDA model using 3 processes
2019-11-02 22:10:58,165 : DEBUG : worker process entering E-step loop
2019-11-02 22:10:58,186 : DEBUG : worker process entering E-step loop
2019-11-02 22:10:58,193 : DEBUG : getting a new job
2019-11-02 22:10:58,196 : DEBUG : worker process entering E-step loop
2019-11-02 22:10:58,201 : DEBUG : getting a new job
2019-11-02 22:10:58,219 : DEBUG : getting a new job
2019-11-02 22:10:58,704 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 

2019-11-02 22:11:05,191 : INFO : topic diff=0.081983, rho=0.462671
2019-11-02 22:11:05,213 : DEBUG : bound: at document #0
2019-11-02 22:11:05,989 : INFO : -11.370 per-word bound, 2646.5 perplexity estimate based on a held-out corpus of 1343 documents with 7958 words
2019-11-02 22:11:06,416 : INFO : PROGRESS: pass 4, dispatched chunk #0 = documents up to #1343/1343, outstanding queue size 1
2019-11-02 22:11:06,734 : DEBUG : processing chunk #0 of 1343 documents
2019-11-02 22:11:06,735 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-02 22:11:07,072 : DEBUG : 1329/1343 documents converged within 50 iterations
2019-11-02 22:11:07,085 : DEBUG : processed chunk, queuing the result
2019-11-02 22:11:07,089 : DEBUG : result put
2019-11-02 22:11:07,093 : DEBUG : getting a new job
2019-11-02 22:11:07,095 : DEBUG : updating topics
2019-11-02 22:11:07,110 : INFO : topic #1 (0.100): 0.002*"image" + 0.002*"model" + 0.002*"word" + 0.002*"network" + 0.002*"text" + 0.001*"learn" + 0

2019-11-02 22:11:14,028 : DEBUG : processing chunk #0 of 1343 documents
2019-11-02 22:11:14,031 : DEBUG : performing inference on a chunk of 1343 documents
2019-11-02 22:11:14,264 : DEBUG : 1339/1343 documents converged within 50 iterations
2019-11-02 22:11:14,274 : DEBUG : processed chunk, queuing the result
2019-11-02 22:11:14,276 : DEBUG : result put
2019-11-02 22:11:14,277 : DEBUG : updating topics
2019-11-02 22:11:14,279 : DEBUG : getting a new job
2019-11-02 22:11:14,285 : INFO : topic #4 (0.100): 0.003*"video" + 0.003*"image" + 0.002*"segmentation" + 0.002*"table" + 0.002*"anomaly" + 0.002*"feature" + 0.002*"pixel" + 0.002*"pose" + 0.002*"network" + 0.001*"frame"
2019-11-02 22:11:14,286 : INFO : topic #3 (0.100): 0.002*"domain" + 0.002*"data" + 0.002*"feature" + 0.002*"model" + 0.001*"target" + 0.001*"temporal" + 0.001*"kernel" + 0.001*"agent" + 0.001*"recommendation" + 0.001*"centrality"
2019-11-02 22:11:14,288 : INFO : topic #8 (0.100): 0.002*"fairness" + 0.002*"planner" + 0.0

2019-11-02 22:11:20,413 : DEBUG : processed chunk, queuing the result
2019-11-02 22:11:20,416 : DEBUG : updating topics
2019-11-02 22:11:20,415 : DEBUG : result put
2019-11-02 22:11:20,418 : DEBUG : getting a new job
2019-11-02 22:11:20,423 : INFO : topic #0 (0.100): 0.002*"dropout" + 0.002*"triplet" + 0.001*"shape" + 0.001*"story" + 0.001*"hypergraph" + 0.001*"personality" + 0.001*"landmark" + 0.001*"end" + 0.001*"scene" + 0.001*"grasp"
2019-11-02 22:11:20,424 : INFO : topic #1 (0.100): 0.002*"message" + 0.002*"emotion" + 0.001*"multilingual" + 0.001*"review" + 0.001*"summarization" + 0.001*"code" + 0.001*"activation" + 0.001*"stackelberg" + 0.001*"distributional" + 0.001*"interpretable"
2019-11-02 22:11:20,425 : INFO : topic #7 (0.100): 0.002*"fairness" + 0.001*"trend" + 0.001*"dictionary" + 0.001*"verification" + 0.001*"minority" + 0.001*"hurricane" + 0.001*"appliance" + 0.001*"ccmtl" + 0.001*"oversampling" + 0.001*"manipulation"
2019-11-02 22:11:20,427 : INFO : topic #5 (0.100): 0.

2019-11-02 22:11:26,515 : DEBUG : updating topics
2019-11-02 22:11:26,516 : DEBUG : getting a new job
2019-11-02 22:11:26,523 : INFO : topic #7 (0.100): 0.001*"fairness" + 0.001*"trend" + 0.001*"dictionary" + 0.001*"minority" + 0.001*"verification" + 0.001*"hurricane" + 0.001*"appliance" + 0.001*"ccmtl" + 0.001*"oversampling" + 0.001*"manipulation"
2019-11-02 22:11:26,524 : INFO : topic #1 (0.100): 0.002*"emotion" + 0.002*"message" + 0.001*"multilingual" + 0.001*"summarization" + 0.001*"activation" + 0.001*"stackelberg" + 0.001*"distributional" + 0.001*"interpretable" + 0.001*"discussion" + 0.001*"stock"
2019-11-02 22:11:26,525 : INFO : topic #3 (0.100): 0.002*"centrality" + 0.002*"saliency" + 0.001*"speaker" + 0.001*"sparsity" + 0.001*"relevance" + 0.001*"segmentation" + 0.001*"circulant" + 0.001*"audio" + 0.001*"metrical" + 0.001*"dnns"
2019-11-02 22:11:26,526 : INFO : topic #2 (0.100): 0.001*"child" + 0.001*"wcsps" + 0.001*"nonverbal" + 0.001*"cognitive" + 0.001*"diffusion" + 0.001*

In [ ]:
for idx, topic in lda_model_tfidf.print_topics(): #num_topics=3, num_words=3
    print('Topic: {}\nWords: {}\n'.format(idx, topic))

In [ ]:
topicWordProbMat_tfidf = lda_model_tfidf.print_topics(num_topics=num_topics,num_words=num_words)
(df_words,DC,zz) = get_chart_data(num_topics,num_words,topicWordProbMat_tfidf)

show_words_table(df_words)
show_dictionary(DC)
show_probs_table(zz)

In [ ]:
show_heapmap(DC,zz)

In [ ]:
file_name = '../models/lda_tfidf_topic_'+ str(num_topics) +'.model'
lda_model_tfidf.save(file_name)